## Import data

### Load

In [3]:
%load_ext autoreload
%autoreload 2

In [17]:
import pandas as pd
import os
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier

In [4]:
# Find the path to the dataset
os.getcwd()

'/Users/leahredon/code/AlcaRmsp/the_laundromat'

In [5]:
# create a csv path
csv_path = '/Users/leahredon/code/AlcaRmsp/the_laundromat/raw_data'
csv_path

'/Users/leahredon/code/AlcaRmsp/the_laundromat/raw_data'

In [6]:
'/Users/leahredon/code/AlcaRmsp/the_laundromat/raw_data'
# Uploading dataset into a dataframe 
df=pd.read_csv(os.path.join(csv_path, 'data.csv'))
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [7]:
# perform one-hot encoding on non-numeric columns
df = pd.get_dummies(df, columns=['type'])

In [8]:
# create a label encoder object
le = LabelEncoder()

# apply the label encoder to non-numeric columns
df['nameOrig'] = le.fit_transform(df['nameOrig'])
df['nameDest'] = le.fit_transform(df['nameDest'])

In [9]:
df.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,757869,170136.0,160296.36,1662094,0.0,0.0,0,0,0,0,0,1,0
1,1,1864.28,2188998,21249.0,19384.72,1733924,0.0,0.0,0,0,0,0,0,1,0
2,1,181.00,1002156,181.0,0.00,439685,0.0,0.0,1,0,0,0,0,0,1
3,1,181.00,5828262,181.0,0.00,391696,21182.0,0.0,1,0,0,1,0,0,0
4,1,11668.14,3445981,41554.0,29885.86,828919,0.0,0.0,0,0,0,0,0,1,0


In [50]:
df = df.drop(['isFlaggedFraud'], axis = 1)

In [ ]:
selected_features

In [51]:
df.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,757869,170136.0,160296.36,1662094,0.0,0.0,0,0,0,0,1,0
1,1,1864.28,2188998,21249.0,19384.72,1733924,0.0,0.0,0,0,0,0,1,0
2,1,181.00,1002156,181.0,0.00,439685,0.0,0.0,1,0,0,0,0,1
3,1,181.00,5828262,181.0,0.00,391696,21182.0,0.0,1,0,1,0,0,0
4,1,11668.14,3445981,41554.0,29885.86,828919,0.0,0.0,0,0,0,0,1,0


In [52]:
estimator = LogisticRegression()
rfe = RFE(estimator, n_features_to_select= 13) # select the top 13 features

In [53]:
X = df.drop('isFraud', axis=1) # predictor variables
y = df['isFraud'] # target variable

In [54]:
rfe.fit(X, y)

RFE(estimator=LogisticRegression(), n_features_to_select=13)

In [61]:
selected_features = rfe.support_
selected_features

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [62]:
# get the ranking array
ranking_features = rfe.ranking_
ranking_features

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [58]:
selected_features

Index(['step', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'type_CASH_IN',
       'type_CASH_OUT', 'type_DEBIT', 'type_PAYMENT', 'type_TRANSFER'],
      dtype='object')

In [64]:
results = pd.DataFrame({'feature': X.columns, 'support': selected_features, 'ranking': ranking_features})
results

,feature,support,ranking
0,step,True,1
1,amount,True,1
2,nameOrig,True,1
3,oldbalanceOrg,True,1
4,newbalanceOrig,True,1
5,nameDest,True,1
6,oldbalanceDest,True,1
7,newbalanceDest,True,1
8,type_CASH_IN,True,1
9,type_CASH_OUT,True,1


Adding flagged fraud back in to test feature selection

In [65]:
df_with_flagged  =pd.read_csv(os.path.join(csv_path, 'data.csv'))
# perform one-hot encoding on non-numeric columns
df_with_flagged = pd.get_dummies(df_with_flagged, columns=['type'])
# create a label encoder object
le = LabelEncoder()

# apply the label encoder to non-numeric columns
df_with_flagged['nameOrig'] = le.fit_transform(df_with_flagged['nameOrig'])
df_with_flagged['nameDest'] = le.fit_transform(df_with_flagged['nameDest'])

In [66]:
df_with_flagged.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,757869,170136.0,160296.36,1662094,0.0,0.0,0,0,0,0,0,1,0
1,1,1864.28,2188998,21249.0,19384.72,1733924,0.0,0.0,0,0,0,0,0,1,0
2,1,181.00,1002156,181.0,0.00,439685,0.0,0.0,1,0,0,0,0,0,1
3,1,181.00,5828262,181.0,0.00,391696,21182.0,0.0,1,0,0,1,0,0,0
4,1,11668.14,3445981,41554.0,29885.86,828919,0.0,0.0,0,0,0,0,0,1,0


In [19]:
df_with_flagged.shape()

NameError: name 'df_with_flagged' is not defined

In [67]:
estimator = LogisticRegression()
rfe = RFE(estimator, n_features_to_select= 14) # select the top 13 features

In [68]:
X = df_with_flagged.drop('isFraud', axis=1) # predictor variables
y = df_with_flagged['isFraud'] # target variable

In [69]:
rfe.fit(X, y)

/Users/leahredon/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RFE(estimator=LogisticRegression(), n_features_to_select=14)

In [70]:
selected_features = rfe.support_
selected_features

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [71]:
# get the ranking array
ranking_features = rfe.ranking_
ranking_features

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [72]:
results = pd.DataFrame({'feature': X.columns, 'support': selected_features, 'ranking': ranking_features})
results

,feature,support,ranking
0,step,True,1
1,amount,True,1
2,nameOrig,True,1
3,oldbalanceOrg,True,1
4,newbalanceOrig,True,1
5,nameDest,True,1
6,oldbalanceDest,True,1
7,newbalanceDest,True,1
8,isFlaggedFraud,True,1
9,type_CASH_IN,True,1


To test if all of the isFlaggedFraud is actually fraud(isFraud = 1)

In [78]:
selected_rows = df_with_flagged[df_with_flagged['isFlaggedFraud'] == 1]
selected_rows

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
2736446,212,4953893.08,5462026,4953893.08,4953893.08,465112,0.0,0.0,1,1,0,0,0,0,1
3247297,250,1343002.08,329539,1343002.08,1343002.08,43638,0.0,0.0,1,1,0,0,0,0,1
3760288,279,536624.41,116746,536624.41,536624.41,29788,0.0,0.0,1,1,0,0,0,0,1
5563713,387,4892193.09,6052892,4892193.09,4892193.09,540034,0.0,0.0,1,1,0,0,0,0,1
5996407,425,10000000.00,5332564,19585040.37,19585040.37,116289,0.0,0.0,1,1,0,0,0,0,1
5996409,425,9585040.37,4552191,19585040.37,19585040.37,32302,0.0,0.0,1,1,0,0,0,0,1
6168499,554,3576297.10,3078566,3576297.10,3576297.10,419584,0.0,0.0,1,1,0,0,0,0,1
6205439,586,353874.22,2248279,353874.22,353874.22,227867,0.0,0.0,1,1,0,0,0,0,1
6266413,617,2542664.27,5651751,2542664.27,2542664.27,471714,0.0,0.0,1,1,0,0,0,0,1
6281482,646,10000000.00,2958273,10399045.08,10399045.08,238527,0.0,0.0,1,1,0,0,0,0,1


Yes all of the isFlaggedFraud, isFraud

In [57]:
df_selected.head()

,amount,isFraud,nameDest,nameOrig,newbalanceDest,newbalanceOrig,oldbalanceDest,oldbalanceOrg,step,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,9839.64,0,1662094,757869,0.0,160296.36,0.0,170136.0,1,0,0,0,1,0
1,1864.28,0,1733924,2188998,0.0,19384.72,0.0,21249.0,1,0,0,0,1,0
2,181.00,1,439685,1002156,0.0,0.00,0.0,181.0,1,0,0,0,0,1
3,181.00,1,391696,5828262,0.0,0.00,21182.0,181.0,1,0,1,0,0,0
4,11668.14,0,828919,3445981,0.0,29885.86,0.0,41554.0,1,0,0,0,1,0


In [43]:
df_ordinal = pd.read_csv(os.path.join(csv_path, 'data.csv'))

In [44]:
df_ordinal.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### Ordinal Encoder

In [46]:
from sklearn.preprocessing import OrdinalEncoder

# Instantiate the Ordinal Encoder
ordinal_encoder = OrdinalEncoder(categories = [['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']])

# Fit it
ordinal_encoder.fit(df_ordinal[["type"]])

# Display the learned categories
display(ordinal_encoder.categories_)

# Transforming categories into ordered numbers
df_ordinal["encoded_type"] = ordinal_encoder.transform(df_ordinal[["type"]])

# Showing the transformed classes
df_ordinal

[array(['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER'],
       dtype=object)]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,encoded_type
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0,3.0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0,3.0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,4.0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0,1.0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0,1.0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0,4.0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0,1.0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0,4.0


In [ ]:
# explore the number of selected features for RFE
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from matplotlib import pyplot
 
# get the dataset
def get_dataset():
 X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
 return X, y
 
# get a list of models to evaluate
def get_models():
 models = dict()
 for i in range(2, 10):
 rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=i)
 model = DecisionTreeClassifier()
 models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
 return models
 
# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
 cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
 scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
 return scores
 
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
 scores = evaluate_model(model, X, y)
 results.append(scores)
 names.append(name)
 print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
# report which features were selected by RFE
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
# define RFE
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
# fit RFE
rfe.fit(X, y)
# summarize all features
for i in range(X.shape[1]):
 print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

### Testing rfecv

In [18]:
# automatically choose the number of features
rfe = RFECV(estimator=DecisionTreeClassifier())

In [ ]:
# automatically select the number of features for RFE
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
# create pipeline
rfe = RFECV(estimator=DecisionTreeClassifier())
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))